[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/vuhung16au/hf-transformer-trove/blob/main/examples/basic1.4/EmissionTracker.ipynb)
[![View on GitHub](https://img.shields.io/badge/View_on-GitHub-blue?logo=github)](https://github.com/vuhung16au/hf-transformer-trove/blob/main/examples/basic1.4/EmissionTracker.ipynb)

# Carbon Emission Tracking for Model Inference with CodeCarbon

## 🎯 Learning Objectives
By the end of this notebook, you will understand:
- Why tracking carbon emissions in ML is important for sustainable AI
- How to use CodeCarbon's EmissionTracker to monitor model inference
- Different patterns for integrating emission tracking in ML workflows
- How to analyze and compare carbon footprints of different models
- Best practices for sustainable machine learning development

## 📋 Prerequisites
- Basic understanding of machine learning concepts
- Familiarity with Hugging Face transformers
- Knowledge of model inference processes
- Understanding of environmental sustainability concepts

## 📚 What We'll Cover
1. **Introduction**: Understanding AI's environmental impact
2. **Setup**: Installing and configuring CodeCarbon
3. **Basic Usage**: Simple emission tracking for model inference
4. **Advanced Patterns**: Context managers and decorators
5. **Model Comparison**: Comparing carbon footprints across models
6. **Analysis**: Visualizing and interpreting emission data
7. **Best Practices**: Guidelines for sustainable ML development
8. **Summary**: Key takeaways and next steps

## 1. Introduction to Carbon Tracking in AI

**Why Track Carbon Emissions in ML?**

Machine learning models, especially large language models, consume significant computational resources, leading to substantial carbon emissions. Understanding and tracking these emissions is crucial for:

- **Environmental Responsibility**: Reducing the carbon footprint of AI systems
- **Cost Optimization**: Energy-efficient models often have lower operational costs
- **Regulatory Compliance**: Meeting environmental reporting requirements
- **Research Insights**: Understanding the relationship between model performance and environmental impact
- **Sustainable Development**: Building AI systems that align with climate goals

**About CodeCarbon**

CodeCarbon is a Python library that tracks the carbon emissions produced by computing workloads. It provides:
- Real-time emission tracking
- Multiple integration patterns (decorators, context managers, explicit tracking)
- Detailed reporting with hardware and location information
- Integration with popular ML frameworks

Let's learn how to use it effectively!

## 2. Setup and Installation

In [ ]:
# Install required packages if not already available
# Uncomment the line below if running in a fresh environment
# !pip install codecarbon>=2.3.0 transformers torch datasets matplotlib seaborn pandas

# Import essential libraries
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import os
from typing import List, Dict, Optional, Tuple
import warnings
warnings.filterwarnings('ignore')

# Hugging Face imports
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    AutoModelForCausalLM, pipeline
)

# CodeCarbon imports
try:
    from codecarbon import EmissionsTracker, track_emissions
    CODECARBON_AVAILABLE = True
    print("✅ CodeCarbon library loaded successfully")
except ImportError:
    CODECARBON_AVAILABLE = False
    print("❌ CodeCarbon not available. Please install with: pip install codecarbon>=2.3.0")
    print("🔄 Continuing with mock implementation for demonstration...")

# Device detection function
def get_device() -> torch.device:
    """
    Get the best available device for PyTorch operations.
    
    Priority order: CUDA > MPS (Apple Silicon) > CPU
    
    Returns:
        torch.device: The optimal device for current hardware
    """
    if torch.cuda.is_available():
        device = torch.device("cuda")
        print(f"🚀 Using CUDA GPU: {torch.cuda.get_device_name()}")
    elif torch.backends.mps.is_available():
        device = torch.device("mps")
        print("🍎 Using Apple MPS for Apple Silicon optimization")
    else:
        device = torch.device("cpu")
        print("💻 Using CPU - consider GPU for better performance")
    
    return device

device = get_device()

print(f"\n📚 Libraries loaded successfully!")
print(f"PyTorch version: {torch.__version__}")
print(f"CodeCarbon available: {'✅' if CODECARBON_AVAILABLE else '❌'}")

## 3. Mock Implementation and Basic Patterns

When CodeCarbon is not available, we'll use a mock implementation to demonstrate the concepts:

In [ ]:
# Mock EmissionTracker for demonstration when CodeCarbon is not available
class MockEmissionsTracker:
    """Mock implementation for demonstration purposes."""
    def __init__(self, project_name=None, experiment_name=None, output_dir=None, output_file=None):
        self.project_name = project_name or "demo_project"
        self.experiment_name = experiment_name or "demo_experiment"
        self.output_dir = output_dir or "./emissions/"
        self.output_file = output_file or "emissions.csv"
        self.start_time = None
        self.emissions = 0.0
        print(f"🎭 Mock EmissionsTracker initialized: {self.project_name}/{self.experiment_name}")
    
    def start(self):
        self.start_time = time.time()
        print(f"🟢 Mock emission tracking started for {self.experiment_name}")
    
    def stop(self):
        if self.start_time:
            duration = time.time() - self.start_time
            # Mock calculation: base emission + complexity factor
            self.emissions = max(0.001, duration * 0.0001)  # Mock emission in kg CO2
            print(f"🛑 Mock emission tracking stopped for {self.experiment_name}")
            print(f"📊 Mock emissions: {self.emissions:.6f} kg CO2")
            return self.emissions
        return 0.0
    
    def __enter__(self):
        self.start()
        return self
    
    def __exit__(self, *args):
        return self.stop()

# Mock decorator
def mock_track_emissions(project_name=None, experiment_name=None, output_dir=None, output_file=None):
    def decorator(func):
        def wrapper(*func_args, **func_kwargs):
            print(f"🎭 Mock emission tracking decorator for function: {func.__name__}")
            start_time = time.time()
            result = func(*func_args, **func_kwargs)
            duration = time.time() - start_time
            mock_emissions = max(0.001, duration * 0.0001)
            print(f"📊 Mock emissions for {func.__name__}: {mock_emissions:.6f} kg CO2")
            return result
        return wrapper
    return decorator

# Use real or mock tracker
TrackerClass = EmissionsTracker if CODECARBON_AVAILABLE else MockEmissionsTracker
emissions_decorator = track_emissions if CODECARBON_AVAILABLE else mock_track_emissions

print("📋 Emission Tracking Patterns Available:")
print("  1. Explicit start/stop tracking")
print("  2. Context manager (with statement)")
print("  3. Decorator pattern")
print("\n🎯 Let's explore each pattern with model inference examples!")

## 4. Load Model for Demonstration

In [ ]:
# Load a small model for demonstration
print("🔄 Loading sentiment analysis model...")
model_name = "distilbert/distilbert-base-uncased-finetuned-sst-2-english"

try:
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    model = model.to(device)
    model.eval()  # Set to evaluation mode
    print(f"✅ Model loaded successfully on {device}")
    
    # Count parameters
    param_count = sum(p.numel() for p in model.parameters())
    print(f"📊 Model parameters: {param_count:,}")
    
    # Test data for inference
    test_texts = [
        "I love this product! It's amazing and works perfectly.",
        "This is terrible. Completely disappointed with the quality.",
        "It's okay, nothing special but not bad either.",
        "Absolutely fantastic! Exceeded all my expectations.",
        "Waste of money. Would not recommend to anyone."
    ]
    
    print(f"📝 Prepared {len(test_texts)} test texts for inference")
    
    MODEL_LOADED = True
    
except Exception as e:
    print(f"❌ Error loading model: {e}")
    print("🔄 This might be due to network issues or memory constraints")
    print("📝 Will continue with demonstration using mock data")
    MODEL_LOADED = False
    test_texts = [
        "Sample positive text for demonstration",
        "Sample negative text for demonstration",
        "Sample neutral text for demonstration"
    ]

## 5. Pattern 1: Explicit Start/Stop Tracking

The most basic pattern where you manually start and stop emission tracking:

In [ ]:
# Pattern 1: Explicit Start/Stop Tracking
print("🎯 Pattern 1: Explicit Start/Stop Emission Tracking")
print("=" * 55)

# Ensure emissions directory exists
os.makedirs('./emissions/', exist_ok=True)

# Initialize the emission tracker
tracker = TrackerClass(
    project_name="hf_sentiment_inference",
    experiment_name="explicit_tracking",
    output_dir="./emissions/",
    output_file="explicit_tracking.csv"
)

# Start tracking
print("\n🟢 Starting emission tracking...")
tracker.start()

# Perform model inference
results = []
inference_start = time.time()

try:
    for i, text in enumerate(test_texts):
        print(f"\n📝 Processing text {i+1}/{len(test_texts)}: {text[:50]}...")
        
        if MODEL_LOADED:
            # Real model inference
            inputs = tokenizer(
                text, 
                return_tensors="pt", 
                padding=True, 
                truncation=True, 
                max_length=512
            )
            inputs = {k: v.to(device) for k, v in inputs.items()}
            
            with torch.no_grad():
                outputs = model(**inputs)
                predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
                predicted_class = torch.argmax(predictions, dim=-1).item()
                confidence = torch.max(predictions).item()
            
            label = "POSITIVE" if predicted_class == 1 else "NEGATIVE"
        else:
            # Mock inference for demonstration
            label = "POSITIVE" if "love" in text.lower() or "fantastic" in text.lower() else "NEGATIVE"
            confidence = 0.85 + np.random.random() * 0.14  # Mock confidence
        
        results.append({
            'text': text,
            'prediction': label,
            'confidence': confidence
        })
        
        print(f"   🎯 Prediction: {label} (confidence: {confidence:.3f})")
        
        # Small delay to simulate processing time
        time.sleep(0.1)

    inference_time = time.time() - inference_start
    print(f"\n⏱️  Total inference time: {inference_time:.2f} seconds")
    
except Exception as e:
    print(f"❌ Error during inference: {e}")
    inference_time = time.time() - inference_start

# Stop tracking and get emissions
print("\n🛑 Stopping emission tracking...")
emissions = tracker.stop()

print(f"\n📊 Emission Tracking Results:")
print(f"   Carbon emissions: {emissions:.6f} kg CO2")
print(f"   Inference time: {inference_time:.2f} seconds")
print(f"   Texts processed: {len(results)}")
if len(results) > 0 and emissions > 0:
    print(f"   Emissions per text: {emissions/len(results):.6f} kg CO2")

## 6. Pattern 2: Context Manager (Recommended)

The context manager pattern is more pythonic and automatically handles start/stop operations:

In [ ]:
# Pattern 2: Context Manager Tracking
print("🎯 Pattern 2: Context Manager Emission Tracking")
print("=" * 55)

# Test with a different batch of texts
batch_texts = [
    "The weather is beautiful today!",
    "I'm feeling quite frustrated with this situation.",
    "This movie was absolutely incredible!",
    "The service was disappointing and slow.",
    "I have mixed feelings about this decision."
]

# Use context manager for automatic tracking
with TrackerClass(
    project_name="hf_sentiment_inference",
    experiment_name="context_manager",
    output_dir="./emissions/",
    output_file="context_manager.csv"
) as tracker:
    
    print(f"\n🔄 Processing {len(batch_texts)} texts with context manager...")
    
    batch_results = []
    batch_start = time.time()
    
    try:
        if MODEL_LOADED:
            # Batch processing for efficiency
            print("\n📦 Using batch processing for better efficiency...")
            
            # Tokenize all texts together
            batch_inputs = tokenizer(
                batch_texts,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=512
            )
            batch_inputs = {k: v.to(device) for k, v in batch_inputs.items()}
            
            # Perform batch inference
            with torch.no_grad():
                batch_outputs = model(**batch_inputs)
                batch_predictions = torch.nn.functional.softmax(batch_outputs.logits, dim=-1)
                predicted_classes = torch.argmax(batch_predictions, dim=-1)
                confidences = torch.max(batch_predictions, dim=-1).values
            
            # Process results
            for i, (text, pred_class, confidence) in enumerate(zip(
                batch_texts, predicted_classes.cpu().numpy(), confidences.cpu().numpy()
            )):
                label = "POSITIVE" if pred_class == 1 else "NEGATIVE"
                batch_results.append({
                    'text': text,
                    'prediction': label,
                    'confidence': confidence
                })
                print(f"   📝 Text {i+1}: {label} ({confidence:.3f}) - {text[:40]}...")
        
        else:
            # Mock processing for demonstration
            print("\n🎭 Using mock processing for demonstration...")
            for i, text in enumerate(batch_texts):
                # Simple sentiment analysis based on keywords
                positive_words = ['beautiful', 'incredible', 'amazing', 'good', 'great']
                negative_words = ['frustrated', 'disappointing', 'slow', 'bad', 'terrible']
                
                text_lower = text.lower()
                if any(word in text_lower for word in positive_words):
                    label = "POSITIVE"
                    confidence = 0.8 + np.random.random() * 0.2
                elif any(word in text_lower for word in negative_words):
                    label = "NEGATIVE"
                    confidence = 0.8 + np.random.random() * 0.2
                else:
                    label = "NEUTRAL"
                    confidence = 0.6 + np.random.random() * 0.3
                
                batch_results.append({
                    'text': text,
                    'prediction': label,
                    'confidence': confidence
                })
                print(f"   📝 Text {i+1}: {label} ({confidence:.3f}) - {text[:40]}...")
                
                # Simulate processing time
                time.sleep(0.05)
        
        batch_time = time.time() - batch_start
        print(f"\n⏱️  Batch processing time: {batch_time:.2f} seconds")
        print(f"⚡ Average time per text: {batch_time/len(batch_texts):.3f} seconds")
        
    except Exception as e:
        print(f"❌ Error during batch processing: {e}")
        batch_results = []

print(f"\n✅ Context manager automatically handled emission tracking!")
print(f"📊 Processed {len(batch_results)} texts successfully")

# Display results summary
if batch_results:
    positive_count = sum(1 for r in batch_results if r['prediction'] == 'POSITIVE')
    negative_count = sum(1 for r in batch_results if r['prediction'] == 'NEGATIVE')
    neutral_count = len(batch_results) - positive_count - negative_count
    avg_confidence = np.mean([r['confidence'] for r in batch_results])
    
    print(f"\n📈 Results Summary:")
    print(f"   Positive: {positive_count}, Negative: {negative_count}, Neutral: {neutral_count}")
    print(f"   Average confidence: {avg_confidence:.3f}")

## 7. Pattern 3: Decorator Pattern

For functions that you want to consistently track, decorators provide a clean solution:

In [ ]:
# Pattern 3: Decorator Pattern
print("🎯 Pattern 3: Decorator Pattern Emission Tracking")
print("=" * 55)

# Define a function with emission tracking
@emissions_decorator(
    project_name="hf_sentiment_inference",
    experiment_name="decorator_pattern",
    output_dir="./emissions/",
    output_file="decorator_pattern.csv"
)
def analyze_sentiment_with_emissions(texts: List[str]) -> List[Dict]:
    """
    Analyze sentiment for a list of texts with automatic emission tracking.
    
    Args:
        texts: List of text strings to analyze
        
    Returns:
        List of dictionaries containing analysis results
    """
    print(f"🔄 Starting sentiment analysis for {len(texts)} texts...")
    
    results = []
    
    try:
        for i, text in enumerate(texts):
            if MODEL_LOADED:
                # Real model inference
                inputs = tokenizer(
                    text,
                    return_tensors="pt",
                    padding=True,
                    truncation=True,
                    max_length=512
                )
                inputs = {k: v.to(device) for k, v in inputs.items()}
                
                with torch.no_grad():
                    outputs = model(**inputs)
                    predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
                    predicted_class = torch.argmax(predictions, dim=-1).item()
                    confidence = torch.max(predictions).item()
                
                label = "POSITIVE" if predicted_class == 1 else "NEGATIVE"
            else:
                # Mock inference
                positive_indicators = ['love', 'great', 'excellent', 'brilliant', 'fantastic']
                negative_indicators = ['hate', 'terrible', 'awful', 'disappointing', 'worst']
                
                text_lower = text.lower()
                if any(word in text_lower for word in positive_indicators):
                    label = "POSITIVE"
                    confidence = 0.85 + np.random.random() * 0.14
                elif any(word in text_lower for word in negative_indicators):
                    label = "NEGATIVE"
                    confidence = 0.85 + np.random.random() * 0.14
                else:
                    # Random prediction for neutral texts
                    label = np.random.choice(["POSITIVE", "NEGATIVE"])
                    confidence = 0.6 + np.random.random() * 0.3
            
            result = {
                'text': text,
                'prediction': label,
                'confidence': confidence,
                'text_length': len(text)
            }
            results.append(result)
            
            print(f"   {i+1:2d}. {label:8s} ({confidence:.3f}) - {text[:50]}...")
            
            # Simulate processing delay
            time.sleep(0.05)
        
        print(f"\n✅ Analysis completed for {len(results)} texts")
        
    except Exception as e:
        print(f"❌ Error during analysis: {e}")
    
    return results

# Test the decorated function
decorator_texts = [
    "I absolutely love using transformers for NLP tasks!",
    "This library is confusing and poorly documented.",
    "CodeCarbon helps make AI development more sustainable.",
    "I'm not sure if this approach will work effectively.",
    "The combination of HuggingFace and emission tracking is brilliant!"
]

print(f"\n🚀 Testing decorated function with {len(decorator_texts)} texts...")
decorator_results = analyze_sentiment_with_emissions(decorator_texts)

print(f"\n📈 Decorator Pattern Results Summary:")
if decorator_results:
    positive_count = sum(1 for r in decorator_results if r['prediction'] == 'POSITIVE')
    negative_count = len(decorator_results) - positive_count
    avg_confidence = np.mean([r['confidence'] for r in decorator_results])
    avg_text_length = np.mean([r['text_length'] for r in decorator_results])
    
    print(f"   Positive predictions: {positive_count}")
    print(f"   Negative predictions: {negative_count}")
    print(f"   Average confidence: {avg_confidence:.3f}")
    print(f"   Average text length: {avg_text_length:.1f} characters")

## 8. Analysis and Best Practices

Let's analyze the emission tracking results and discuss best practices:

In [ ]:
# Analysis and Visualization of Emission Data
print("📊 Carbon Emission Analysis and Best Practices")
print("=" * 55)

# Create mock comparison data for demonstration
experiment_data = [
    {'pattern': 'Explicit Tracking', 'texts': 5, 'time': 2.1, 'emissions': 0.001234},
    {'pattern': 'Context Manager', 'texts': 5, 'time': 1.8, 'emissions': 0.000987},
    {'pattern': 'Decorator', 'texts': 5, 'time': 2.3, 'emissions': 0.001156}
]

# Create DataFrame for analysis
df = pd.DataFrame(experiment_data)
df['Efficiency (texts/kg CO2)'] = df['texts'] / df['emissions']
df['Time per Text (s)'] = df['time'] / df['texts']
df['Emissions per Text (kg CO2)'] = df['emissions'] / df['texts']

print("\n📋 Experiment Comparison Summary:")
print(df.round(6))

# Create visualizations
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Carbon Emission Tracking Pattern Analysis', fontsize=16, fontweight='bold')

# 1. Emissions comparison
bars1 = axes[0, 0].bar(df['pattern'], df['emissions'], color=['lightcoral', 'lightblue', 'lightgreen'])
axes[0, 0].set_ylabel('Total Emissions (kg CO2)')
axes[0, 0].set_title('Total Emissions by Pattern')
axes[0, 0].tick_params(axis='x', rotation=45)

# Add value labels on bars
for bar in bars1:
    height = bar.get_height()
    axes[0, 0].text(bar.get_x() + bar.get_width()/2., height,
                   f'{height:.6f}',
                   ha='center', va='bottom', fontsize=10)

# 2. Efficiency comparison
bars2 = axes[0, 1].bar(df['pattern'], df['Efficiency (texts/kg CO2)'], 
                      color=['lightcoral', 'lightblue', 'lightgreen'])
axes[0, 1].set_ylabel('Texts per kg CO2')
axes[0, 1].set_title('Carbon Efficiency by Pattern')
axes[0, 1].tick_params(axis='x', rotation=45)

for bar in bars2:
    height = bar.get_height()
    axes[0, 1].text(bar.get_x() + bar.get_width()/2., height,
                   f'{height:.0f}',
                   ha='center', va='bottom', fontsize=10)

# 3. Processing time comparison
bars3 = axes[1, 0].bar(df['pattern'], df['Time per Text (s)'], 
                      color=['lightcoral', 'lightblue', 'lightgreen'])
axes[1, 0].set_ylabel('Time per Text (seconds)')
axes[1, 0].set_title('Processing Efficiency by Pattern')
axes[1, 0].tick_params(axis='x', rotation=45)

for bar in bars3:
    height = bar.get_height()
    axes[1, 0].text(bar.get_x() + bar.get_width()/2., height,
                   f'{height:.3f}',
                   ha='center', va='bottom', fontsize=10)

# 4. Emissions vs Time scatter plot
scatter = axes[1, 1].scatter(df['Time per Text (s)'], df['Emissions per Text (kg CO2)'], 
                           s=100, c=['red', 'blue', 'green'], alpha=0.7)
axes[1, 1].set_xlabel('Time per Text (seconds)')
axes[1, 1].set_ylabel('Emissions per Text (kg CO2)')
axes[1, 1].set_title('Time vs Emissions Efficiency')

# Add pattern labels
for i, row in df.iterrows():
    axes[1, 1].annotate(row['pattern'], 
                       (row['Time per Text (s)'], row['Emissions per Text (kg CO2)']),
                       xytext=(5, 5), textcoords='offset points',
                       fontsize=9)

plt.tight_layout()
plt.show()

# Analysis insights
print("\n🔍 Key Insights:")
most_efficient = df.loc[df['Efficiency (texts/kg CO2)'].idxmax()]
fastest = df.loc[df['Time per Text (s)'].idxmin()]

print(f"   🏆 Most carbon-efficient: {most_efficient['pattern']}")
print(f"      {most_efficient['Efficiency (texts/kg CO2)']:.0f} texts per kg CO2")
print(f"   ⚡ Fastest processing: {fastest['pattern']}")
print(f"      {fastest['Time per Text (s)']:.3f} seconds per text")

efficiency_diff = df['Efficiency (texts/kg CO2)'].max() / df['Efficiency (texts/kg CO2)'].min()
print(f"   📈 Efficiency range: {efficiency_diff:.1f}x difference between best and worst")

## 9. Best Practices for Sustainable ML

Based on our emission tracking experiments, here are key recommendations:

In [ ]:
# Best Practices for Sustainable ML
print("🌱 Best Practices for Sustainable Machine Learning")
print("=" * 60)

best_practices = {
    "🎯 Model Selection": [
        "Choose the smallest model that meets your accuracy requirements",
        "Consider distilled models (like DistilBERT) for production deployment",
        "Evaluate trade-offs between model performance and carbon footprint",
        "Use model cards to understand environmental impact information"
    ],
    
    "⚡ Optimization Strategies": [
        "Use batch processing instead of individual predictions",
        "Implement model quantization and pruning techniques",
        "Leverage mixed precision training (FP16) when possible",
        "Cache model outputs for repeated inputs",
        "Use ONNX or TensorRT for optimized inference"
    ],
    
    "🏗️ Infrastructure Choices": [
        "Choose cloud regions with renewable energy sources",
        "Prefer CPU inference for small models and low latency requirements",
        "Use auto-scaling to minimize idle compute resources",
        "Consider edge deployment to reduce data transfer"
    ],
    
    "📊 Monitoring and Reporting": [
        "Track carbon emissions throughout the ML lifecycle",
        "Set carbon budgets for ML projects",
        "Include environmental impact in model evaluation metrics",
        "Report emissions alongside traditional performance metrics"
    ],
    
    "🔄 Development Workflow": [
        "Use smaller models or datasets during development and debugging",
        "Implement early stopping to avoid unnecessary training",
        "Share pre-trained models to avoid redundant training",
        "Consider federated learning to reduce centralized computation"
    ],
    
    "📋 CodeCarbon Integration": [
        "Use context managers for clean, automatic tracking",
        "Apply decorators to functions that need consistent monitoring",
        "Configure appropriate project and experiment names",
        "Store emission logs for historical analysis"
    ]
}

for category, practices in best_practices.items():
    print(f"\n{category}")
    for i, practice in enumerate(practices, 1):
        print(f"   {i}. {practice}")

print("\n" + "=" * 60)
print("💡 Key Takeaway: Sustainable AI is about making informed trade-offs")
print("   between model performance, computational efficiency, and environmental impact.")
print("   Always measure and optimize for the right balance for your use case.")

# Environmental context
print(f"\n🌍 Environmental Impact Context:")
mock_total_emissions = sum(exp['emissions'] for exp in experiment_data)
print(f"   Total emissions from our experiments: {mock_total_emissions:.6f} kg CO2")
print(f"   Equivalent to approximately {mock_total_emissions * 1000:.2f} grams of CO2")

# Rough equivalencies for context
smartphone_charges = (mock_total_emissions * 1000) / 8  # ~8g CO2 per smartphone charge
driving_meters = (mock_total_emissions * 1000) / 404 * 1000  # ~404g CO2 per km

print(f"   Equivalent to ~{smartphone_charges:.2f} smartphone charges")
print(f"   Equivalent to ~{driving_meters:.1f} meters of car driving")

print(f"\n🎯 Action Items:")
print(f"   1. Integrate CodeCarbon into your ML workflows")
print(f"   2. Set carbon budgets for your projects")
print(f"   3. Compare models based on efficiency, not just accuracy")
print(f"   4. Share emission data with your team and community")
print(f"   5. Choose sustainable infrastructure and deployment options")

## 10. Summary

In this comprehensive notebook, we explored carbon emission tracking for ML model inference:

### 🔑 **Key Concepts Mastered**
- **CodeCarbon Integration**: Multiple patterns for tracking emissions (explicit, context manager, decorator)
- **Model Inference Monitoring**: Real-time tracking of carbon footprint during model inference
- **Pattern Comparison**: Understanding trade-offs between different tracking approaches
- **Emission Analysis**: Visualizing and interpreting carbon emission data
- **Sustainable ML**: Best practices for environmentally responsible AI development
- **Mock Implementation**: Demonstration techniques when libraries aren't available

### 📈 **Best Practices Learned**
- Use context managers for clean emission tracking code
- Apply decorators for functions requiring consistent monitoring
- Implement batch processing to reduce emissions per prediction
- Choose the smallest model that meets accuracy requirements
- Monitor and report emissions alongside traditional ML metrics
- Consider environmental impact in model selection and deployment decisions
- Set carbon budgets for ML projects

### 🚀 **Next Steps**
- **Production Integration**: Implement emission tracking in production ML pipelines
- **Advanced Optimization**: Explore model quantization and pruning for efficiency
- **Green Computing**: Learn about renewable energy sources for ML workloads
- **Community Sharing**: Contribute emission data to the ML community
- **Policy Development**: Create organizational guidelines for sustainable AI

### 💡 **Key Takeaway**
Sustainable AI development requires balancing model performance with environmental impact. By tracking carbon emissions during model inference, we can make informed decisions that contribute to more responsible and environmentally conscious machine learning practices. CodeCarbon provides the tools to measure our impact, and it's our responsibility to act on that information.

---

## About the Author

**Vu Hung Nguyen** - AI Engineer & Researcher

Connect with me:
- 🌐 **Website**: [vuhung16au.github.io](https://vuhung16au.github.io/)
- 💼 **LinkedIn**: [linkedin.com/in/nguyenvuhung](https://www.linkedin.com/in/nguyenvuhung/)
- 💻 **GitHub**: [github.com/vuhung16au](https://github.com/vuhung16au/)

*This notebook is part of the [HF Transformer Trove](https://github.com/vuhung16au/hf-transformer-trove) educational series.*